In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics

plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# used for Alexnet
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
# define samplers for obtaining training and validation batches
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
model_short_names = ['resnet18', 'vgg']
framework = 'pytorch'
ext = '.pth'
def validate(model, data):
    total = 0
    correct = 0
    since = time.time()
    for i, (images, labels) in enumerate(data):
        #images = images.cuda()
        x = model(images)
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        total += x.size(0)
        correct += torch.sum(pred == labels)
    time_elapsed = time.time() - since
    #print('accuracy: {}%'.format(correct*100./total),'time {:.0f}m {:.0f}s, {}'.format(time_elapsed // 60, time_elapsed % 60, time_elapsed) )
    #return correct*100./total
    return correct*100./total, time_elapsed
def run_prediction():
    for model_short_name in model_short_names:
        #for model in models: 
        data_accuracy = []
        data_time = []
        path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}'.format(framework,model_short_name)
        if os.path.exists(path):
            list_files = [x for x in os.listdir(path) if ext in x]
            for i in range(1, 11):
                for model_name in list_files:
                    if '_{}{}'.format(i, ext) in model_name:
                        model = torch.load(path+'/'+model_name, map_location=torch.device('cpu'))
                        accuracy, time_ = validate(model, valloader)
                        data_accuracy.append(accuracy)
                        data_time.append(time_)
        print('Avg accuracy: {}, avg time: {}'.format(np.mean(data_accuracy), np.mean(data_time)))
run_prediction()

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [6]:
training_id =10
model_short_name = 'resnet18'
framework = 'pytorch'

In [7]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/pytorch/{}/'.format(model_short_name)

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_exp_resnet_2021-11-06_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

44813453

In [8]:
import numpy as np
def validate(model, data):
    total = 0
    correct = 0
    list_time = []
    for i, (images, labels) in enumerate(data):
        #images = images.cuda()
        since = time.time()
        x = model(images)
        
        
        
        value, pred = torch.max(x,1)
        pred = pred.data.cpu()
        time_elapsed = time.time() - since
        print(i, time_elapsed)
        total += x.size(0)
        correct += torch.sum(pred == labels)
        list_time.append(time_elapsed)
    print('accuracy: {}%'.format(correct*100./total),np.mean(list_time) )
    #return correct*100./total

In [9]:
validate(model, valloader)

0 0.6254169940948486
1 0.55877685546875
2 0.47291111946105957
3 0.6337087154388428
4 0.6841578483581543
5 0.6307680606842041
6 0.6682338714599609
7 0.5347340106964111
8 0.6343228816986084
9 0.48595285415649414
10 0.4507420063018799
11 0.4459071159362793
12 0.6943769454956055
13 0.5767428874969482
14 0.47185420989990234
15 0.4573211669921875
16 0.43680620193481445
17 0.48111891746520996
18 0.777540922164917
19 0.5027899742126465
20 0.4551229476928711
21 0.4766530990600586
22 0.5620858669281006
23 0.439863920211792
24 0.4508249759674072
25 0.5012998580932617
26 0.5971047878265381
27 0.5486140251159668
28 0.44445180892944336
29 0.43605804443359375
30 0.48169589042663574
31 0.4302670955657959
32 0.4600651264190674
33 0.5332880020141602
34 0.44472599029541016
35 0.42999696731567383
36 0.459320068359375
37 0.4360032081604004
38 0.43382692337036133
39 0.4620389938354492
40 0.42690205574035645
41 0.43530797958374023
42 0.43753695487976074
43 0.4251430034637451
44 0.46532416343688965
45 0.55077

In [10]:
training_id =10
model_short_name = 'vgg'
framework = 'pytorch'

In [11]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)

# Model class must be defined somewhere
since_0 = time.time()
model_name = 'torch_exp_vgg_2021-11-04_{}'.format(training_id)
model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
#resnet50_model.eval()
t_elapsed_0 = time.time() - since_0
size0 = os.path.getsize(path+model_name+'.pth')
size0

515298923

In [12]:
validate(model, valloader)

0 2.2228879928588867
1 3.032788038253784
2 1.989717960357666
3 1.990067958831787
4 1.8611130714416504
5 1.6754341125488281
6 2.4087018966674805
7 2.1932668685913086
8 1.792358160018921
9 1.6843540668487549
10 1.700124979019165
11 1.6505098342895508
12 1.6222169399261475
13 2.0847232341766357
14 1.903141736984253
15 1.8299281597137451
16 2.589092969894409
17 2.7819278240203857
18 2.2879862785339355
19 2.420562267303467
20 2.1776199340820312
21 2.477008104324341
22 2.1414310932159424
23 2.808889150619507
24 2.220054864883423
25 2.378525972366333
26 2.3863179683685303
27 2.333230972290039
28 2.9359118938446045
29 3.0750882625579834
30 2.767108201980591
31 2.2587382793426514
32 2.3404290676116943
33 3.188523054122925
34 2.7022058963775635
35 2.5056328773498535
36 2.9587371349334717
37 2.962963342666626
38 3.1609508991241455
39 2.9019830226898193
40 2.9440457820892334
41 2.5935301780700684
42 2.826900005340576
43 2.7577919960021973
44 3.3551809787750244
45 3.5112991333007812
46 3.1055140495